# Front End Lab 3: Aliasing and Anti-Aliasing Filter

### EE 16B: Designing Information Devices and Systems II, Fall 2015

**Name 1**:

**Login**: ee16b-


**Name 2**:

**Login**: ee16b-


##Table of Contents

* [Introduction](#intro)
* [Task 0: Lab 2 Setup](#task0)
* [Task 1: Aliasing](#task1)
* [Task 2: Anti-Aliasing Filter](#task2)

<a id='intro'></a>
## Introduction

The block diagram below shows our circuit from last week. It amplifies the input voltage with a large gain and takes care of the voltage offset and voltage range of the ADC.

<img style="width: 750px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-block-3.png">

As we will seen in lecture, it is very useful to see how our signal behaves in the frequency domain. When sampling a signal, we must make sure to take samples frequently enough to capture the major events. However, our Launchpads can only sample so fast - we will see the effects of a finite sample rate in Task 1.

<a id='task0'></a>
##<span style="color:blue">Task 0: Lab 2 Setup</span>

Last week we modified our cascade of 3 amplifiers to include a DC Reject and a level-shift amplifier. Test each block and make sure they still work as expected before continuing with this lab.

This week, we will use the function generators to produce one of the BMI waveforms we played with earlier in the course. To load the waveform:
1. Select the <b>Arb</b> button
2. Select Select <b>Wform</b>
3. Select <b>Stored Wforms</b>
4. Find EE16BFE in Arb Mem 1,2,3 or 4 and select <b>SELECT ARB</b>
5. Set <b>High Z</b> (Utility > Output Setup > High Z)
6. Set <b>Freq</b> to <b>100mHz</b>
7. Set <b>Amplitude</b> to <b>100mVpp</b>
8. Turn on Output

**<span style="color:red">Check the output of your frontend (not function generator) is within 0~3.3V range. Too high/low voltage will break your MSP! </span>**

<a id='task1'></a>
##<span style="color:blue">Task 1: Aliasing</span>

###Materials
- Lab 2 setup

Let's take a look at the frequency content of our input signal. The data in the function generator was uploaded from the file `notMoving.csv`. Note that the plot below only shows the input signal - it does not show the noise added by the signal processing circuit.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = np.genfromtxt('notMoving.csv', delimiter=',')[:,1]
plt.plot(data)
plt.title('notMoving.csv')
plt.xlabel('Samples')
plt.ylabel('Magnitude')
data_fft = np.fft.rfft(data);
plt.figure()
plt.plot(np.linspace(0,500,len(data_fft)), abs(data_fft))
plt.title('FFT of notMoving.csv')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')

As we continue forward, keep in mind that the Launchpad will sample at <b> 333 Hz </b>. You don't need to know what the effect of this rate, but you might intuitively guess what will happen to frequencies greater than 333 Hz. To explore what will happen, lets hook up our Launchpads.

We will be using the <a href="http://www.ti.com/ww/en/launchpad/launchpads-msp430-msp-exp430f5529lp.html#tabs">TI MSP430F5529 Launchpad</a> as an ADC for our this lab, and later for our projects. The development board includes the <a href="http://www.ti.com/lit/ds/symlink/msp430f5514.pdf">MSP430F5529 microcontroller</a>, which we used last semester in 16A.

<img style="width: 400px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/msp430.png">

The Launchpad interfaces with your PC through a USB cable and you can program the microcontroller through a couple different software, but we will use Energia.

Let's start loading some code on the Launchpad. Much of the following will be review from 16A, but read through it for a quick refresher.

Launch the Energia software and choose "Launchpad w/ msp430f5529 (25MHz)" under `Tools > Board`. You need to make sure to choose one of the msp430f5529's when working with these boards. The number in the parenthesis is the clock speed you want the microcontroller to run at.

<img style="width:500px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/msp-setboard.png">

When you connect the Launchpad to the PC using the USB cable, the Launchpad starts 2 COM ports to communicate with the PC. One of these is used for the Application UART (Universal Asynchronous Receiver/Transmitter) while the other is used for the debug interface. To choose the right COM port to connect to, click the Windows Home button and type in "Device Manager". Then, click "Ports (COM & LPT)". Note the COM value of the Application UART. <b>You need to do this every time you connect a Launchpad to a PC because the COM port might change.</b>

<img style="width:400px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/msp-devicemanager.png">

Under `Tools > Serial Port` choose the COM port you have noted before.

<img style="width:400px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/msp-setport.png">

Now click `File > Examples > Basics > Blink`. This opens an example sketch (code) that blinks one of the LEDs on the Launchpad.

<img style="width:400px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/msp-blink.png">

Click the Upload button (the right arrow in the top left of the window). If Energia asks for a firmware update, follow the instruction by clicking `Tools > Update Programmer`. Then, click on the upload button again.

<img style="width:400px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/msp-upload.png">

<img style="width:400px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/msp-firmwareupdate.png">

Press the reset button (`RST`) in the Launchpad. The red LED on your Launchpad should blink. If it doesn't, get a GSI or lab helper to help you debug. In general, always hit the reset button after every upload.

Now, upload the sketch <b>`adc_read.ino`</b> to your Launchpad - make sure the board is set to MSP430F5529 (25MHz) in Energia. Open Device Manager to find the COM port of the Application UART, then hit the reset button on the Launchpad. Connect pin `P6.0` to the output of the level-shift amplifier and the ground of the Launchpad to the ground of the circuit. Then run the cell below, choose a right COM port, press RST on your MSP to gather 10 seconds of data. The code then takes the DFT of the collected data and plots the signal in the frequency domain. <b>It will take a while to run </b> 

Hint: It usually takes ~1 miniute and if it's not finished after 2 minutes, restart your Kernel, open cmd, and type <b>python adc_run.py</b> and follow instructions. It will show more internal messages to check if it's running. Be sure to press RST after you choose the serial port.

In [ ]:
from adc_record import *
run('adc_before.txt')

Run this to load the recorded data

In [ ]:
lines = list(open('adc_before.txt'))  
samples = [int(x)/4096 for x in lines]
samples = samples - np.mean(samples) # Normalizing to center at 0
plt.plot(samples)
plt.title('Recorded Signal')
plt.xlabel('Samples')
plt.ylabel('Magnitude')

In [ ]:
before_fft = np.fft.rfft(samples);

plt.plot(np.linspace(0,333/2,len(before_fft)), abs(before_fft))
plt.title('FFT of ADC output')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.xlim(0,333/2)

Yikes! That's very different from what we expect! We can see that the largest frequency present in the frequency plot from the ADC is 166.5 Hz, or $f_s$/2 (333/2 Hz). Remember this - it just might show up in lecture later.

But even in this smaller domain, the output doesn't match the frequency plot we found above: tehre are some added frequencies in there too! This is the result of a phenomenon called *aliasing* which will be covered in lecture in the upcoming weeks. For the remainder of this lab, you don't need to understand what aliasing is, just that it happens because we have signals with too high a frequency for our sampling rate.

So what can we do? We can add a filter of course! 

<a id='task2'></a>
##<span style="color:blue">Task 2: Anti-Aliasing Filter</span>

###Materials
- Task 1 setup
- 1 10nF capacitor

Lucky for us, we can remove both aliasing and high frequency noise using an anti-aliasing filter. We are going to modify the first non-inverting amplifier we had after the DC reject circuit to an active filter circuit below.

<img style="width:350px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-lpf.png">

This circuit will both filter the signal and apply some gain.

Let's conceptually explore the active lowpass filter above. First, consider the feedback loop with *both* the capacitor and resistor.

**<span style="color:red">How does the capacitor function at small frequencies (DC)? What will the output voltage be?**

YOUR ANSWER HERE

**<span style="color:red">How does the capacitor function at high frequencies? What will this do to the current?</span>**

YOUR ANSWER HERE

**<span style="color:red">What kind of filter is this?</span>**

YOUR ANSWER HERE

Now modify your first non-inverting amplifier to the active low-pass circuit above. Note that the signal is now passed to the <b> inverting </b> terminal of the op-amp. Use the values:
- $R_1$ = 10k $\Omega$
- $R_2$ = 100k $\Omega$
- $C$ = 10nF

Your final circuit block diagram should look like below:
<img style="width: 750px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-block.png">

##<span style="color:green">BUT WAIT!</span>

Before you continue, <b>there is a problem: the active low pass circuit has a finite input resistance!</b> This means it gives some loading to the stage right before it, the DC reject circuit in this case. If we leave it like this, the DC reject cutoff frequency will be shifted since the effective resistance of the filter is changed (recall that we calculated the 3dB frequency using both R and C. Adding a load to the reject circuit will change the R, and therefore the cutoff frequency).

**<span style="color:red">What can we add between the DC Reject and the Active Low-Pass Filter to solve this issue?</span>** Hint: we have seen a simple circuit that isolates resistive loads. Check with a GSI or lab assistant before you continue.

YOUR ANSWER HERE

Now add the circuit you suggest above to solve the problem.

Probe both the function generator signal and the final output of the circuit - you will see that the amplitude gets reduced, but the signal is very clean compared to before.

<img style="width:600px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-scope-LPF.png">

**<span style="color:red">Why is the output different from before? How does this filter effectively reduce aliasing?</span>**

YOUR ANSWER HERE

Now try running the cells below, which records 10 seconds of samples and plots the DFT of the signal just like before. Compare the DFT plot with the original signal.

In [ ]:
from adc_record import *
run('adc_after.txt')

In [ ]:
lines = list(open('adc_after.txt'))  
samples = [int(x)/4096 for x in lines]
samples = samples - np.mean(samples) # Normalizing to center at 0
plt.plot(samples)
plt.title('Recorded Signal')
plt.xlabel('Samples')
plt.ylabel('Magnitude')

In [ ]:
after_fft = np.fft.rfft(samples);

plt.plot(np.linspace(0,333/2,len(after_fft)), abs(after_fft))
plt.title('FFT of ADC output')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.xlim(0,333/2)

plt.figure()
plt.plot(np.linspace(0,500,len(data_fft)), abs(data_fft))
plt.xlim(0,333/2)
plt.title('FFT of notMoving.csv')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')

How does it look? Satisfied?

One thing to note is the output range of the low-pass filter: it is quite a bit smaller than the range of -200mV to 200mV that we had assumed earlier. This is due to the fact that we have removed many of the frequencies of the signal with the active lowpass filter, and thus have reduced the overall power.

Congratulations - you've completed the whole front-end chain that amplifies $\mu V$-amplitude signals to a range that is processable by the Launchpad ADC! You built a cascade of amplifiers and added both a DC reject and level shift, and solved the aliasing and high frequency noise problem with an active low-pass filter.